In [39]:
print("Analyse Bourse")

Analyse Bourse


#Let's go learning PYTHON !

In [40]:
#Ceci est un commentaire dans la cellule de code
#Python est de la programmation OBJET. On défini des variables.
name = "Corentin"

print(name)

Corentin


In [41]:
name
#Si on appelle la variable alors on voit son type.


'Corentin'

In [42]:
#Pour les chiffres on a 
#Les entiers naturel 
age = 24
print(age)


24


In [43]:
#Pour les chiffres à virgules on parle de FLOATS
pi = 3.14159
pi

3.14159

In [44]:
#Mais nous sommes humains, donc nous LISONS des PHRASES.
#Dès lors, on utilise la fonction PRINT F, qui permet d'avoir un print plus proche de notre language
#Les variables sont en DYNAMIQUES, 
#print(f'Hello, les boloss dont {name}')

In [45]:
is_cool = True
is_cool

True

#Présentation des DATA STRUCTURES

In [46]:
#Les LISTES se nomment list en anglais
stocks = {"AAPL","GOOGL","MSFT","TSLA"}
stocks

{'AAPL', 'GOOGL', 'MSFT', 'TSLA'}

In [47]:
#On peut ajouter des inputs dans la liste via la fonction APPEND
stocks.append("AMZN")
stocks

AttributeError: 'set' object has no attribute 'append'

In [ ]:
#UN DICTIONNAIRE c'est mieux encore qu'une liste
# On parle de Clé-Valeur
prices = {
    "APPL":248.04,
    "GOOGL":328.33
    "TSLA":437.43
}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (973026087.py, line 5)

In [ ]:
#C'est le fait que PYTHON repose sur des PACKAGES 
#Donc on gagne un temps fou à ne pas ré inventer la ROUE à chaque fois
#! pip install yfinance


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 61.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 78.8 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 76.9 MB/s  0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15637 sha256=bb94cd3f362dae090814456dcbff88eb4ad99941f4ed9e0c7e821419a53b2d04
  Stored in directory: /home/codespace/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
Successfully built multitasking
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [yfinance]/10 [curl_cffi]]


In [50]:
import yfinance as yf